## DIA 115: Permitir que el Usuario Elimine Reportes de su Historial Manualmente

Hoy vas a permitir que el usuario pueda eliminar reportes que ya no necesita directamente desde su historial.

✅ ¿Qué aprenderás hoy?

🗑️ Crear una ruta de eliminación segura en Flask

🛡️ Verificar que el reporte sea del usuario logueado

🖱️ Añadir un botón "Eliminar" junto a cada reporte en la tabla

🧩 1. Crear la ruta para eliminar un reporte
python
Copiar
Editar
@app.route('/eliminar-reporte/<int:reporte_id>', methods=['POST'])
@login_required
def eliminar_reporte(reporte_id):
    reporte = Reporte.query.get_or_404(reporte_id)

    # Verificar que el reporte pertenezca al usuario actual
    if reporte.user_id != current_user.id:
        return "No tienes permiso para eliminar este reporte.", 403

    db.session.delete(reporte)
    db.session.commit()

    return redirect('/historial-reportes')
🧩 2. Modificar el template historial_reportes.html
Agrega una nueva columna Eliminar en tu tabla:

html
Copiar
Editar
<table>
  <thead>
    <tr>
      <th>Fecha</th>
      <th>Tipo</th>
      <th>Enlace</th>
      <th>Eliminar</th>
    </tr>
  </thead>
  <tbody>
    {% for reporte in reportes %}
    <tr>
      <td>{{ reporte.fecha_generado.strftime('%Y-%m-%d %H:%M') }}</td>
      <td>{{ reporte.tipo.upper() }}</td>
      <td><a href="{{ reporte.url }}" target="_blank">📥 Descargar</a></td>
      <td>
        <form method="POST" action="/eliminar-reporte/{{ reporte.id }}" onsubmit="return confirm('¿Seguro que quieres eliminar este reporte?');">
          <button type="submit" style="background-color:red; color:white;">🗑️</button>
        </form>
      </td>
    </tr>
    {% else %}
    <tr><td colspan="4">No hay reportes generados aún.</td></tr>
    {% endfor %}
  </tbody>
</table>
🧩 3. (Opcional) Borrar también el archivo de Firebase (si quieres)
Si quieres ser más limpio, puedes agregar también una función que borre el archivo de Firebase al eliminar el reporte:

python
Copiar
Editar
def borrar_de_firebase(ruta_firebase):
    try:
        storage.delete(ruta_firebase)
        print("✅ Archivo eliminado de Firebase")
    except Exception as e:
        print(f"❌ Error al eliminar archivo: {e}")
Y dentro de eliminar_reporte antes de db.session.delete(reporte), podrías llamar a esta función (si guardaste la ruta Firebase del archivo).

